<a href="https://colab.research.google.com/github/yj5x/Fine-Tuninng-Pre-Trained-Vision-models-PyTorch/blob/master/Task7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikeras
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d snap/amazon-fine-food-reviews -f Reviews.csv --unzip

In [ ]:
# استيراد المكتبات المطلوبة
!pip install scikeras

import os
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import re

# تنزيل أحدث إصدار من مجموعة البيانات
path = kagglehub.dataset_download("snap/amazon-fine-food-reviews")
print("Path to dataset files:", path)


In [ ]:

# تحميل البيانات
data = pd.read_csv(os.path.join(path, "Reviews.csv"))

# استكشاف البيانات
# عدد الصفوف والأعمدة
print("عدد الصفوف والأعمدة:", data.shape)

print()
print()

# أول 5 صفوف من البيانات
display(data.head())

print()
print()

# نظره على أنواع البيانات والقيم المفقوده
data.info()

print()
print()

# نظره على محتوى كل عمود
for col in data.columns:
    display(f"{col}: {data[col].unique()[:5]}")
    print()
    print()

In [ ]:
import re
# تحضير البيانات
data['label'] = data['Score'].apply(lambda x: 1 if x >= 4 else 0)  # 1 للإيجابية، 0 للسلبية
data = data[data['Score'] != 3]  # حذف مراجعات الثلاث نجوم

# تنظيف البيانات
def clean_text(text):
    # إزالة علامات الترقيم
    text = re.sub(r'[^\w\s]', '', text)
    # استبدال الأرقام
    text = re.sub(r'\d+', '#', text)
    # تحويل النص إلى lowercase
    return text.lower()

data['Text'] = data['Text'].apply(clean_text)

# معالجة النص
X = data['Text'].values
y = data['label'].values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, padding='post')

In [ ]:
# تقليص طول المدخلات
max_length = 100  # أو 150 حسب الحاجة
X_pad = pad_sequences(X_seq, maxlen=max_length, padding='post')

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# بناء النموذج
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_length))

# بناء النموذج
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=X_pad.shape[1]))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# تجميع النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# تدريب النموذج
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


In [ ]:

# تقييم النموذج
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# طباعة تقرير التصنيف
print(classification_report(y_test, y_pred_classes))

# مصفوفة الارتباك
conf_matrix = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# الحصول على التوقعات الاحتمالية
y_pred_prob = model.predict(X_test).ravel()  # استخدام .ravel() لتحويلها إلى مصفوفة أحادية

# حساب TPR وFPR
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)  # حساب المساحة تحت المنحنى

# رسم منحنى ROC
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()